In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
import random 

In [3]:
df = pd.read_csv("web-Google_10k.txt", sep=' ')

C:\Users\HARSHA\AppData\Local\Temp\ipykernel_40660\1725221542.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("web-Google_10k.txt", sep=' ')


In [4]:
df

,#,Directed,graph,(each,unordered,pair,of,nodes,is,saved,once):,web-Google.txt,Unnamed: 12
0,#,Sample,10k,pages,of,the,Webgraph,from,the,Google,programming,"contest,",2002.0
1,#,Nodes:,10000,Edges:,78323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#,FromNodeId\tToNodeId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0\t11342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0\t824020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78321,495600\t271199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78322,495600\t407927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78323,495600\t555924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78324,495600\t628974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df=df[['#']]

In [6]:
df

,#
0,#
1,#
2,#
3,0\t11342
4,0\t824020
...,...
78321,495600\t271199
78322,495600\t407927
78323,495600\t555924
78324,495600\t628974


In [7]:
df[['FromNodeId','ToNodeId']] = df['#'].str.split('\t',expand=True)

C:\Users\HARSHA\AppData\Local\Temp\ipykernel_40660\3743569878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['FromNodeId','ToNodeId']] = df['#'].str.split('\t',expand=True)
C:\Users\HARSHA\AppData\Local\Temp\ipykernel_40660\3743569878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['FromNodeId','ToNodeId']] = df['#'].str.split('\t',expand=True)


In [8]:
df=df[['FromNodeId','ToNodeId']]
df

,FromNodeId,ToNodeId
0,#,None
1,#,None
2,#,None
3,0,11342
4,0,824020
...,...,...
78321,495600,271199
78322,495600,407927
78323,495600,555924
78324,495600,628974


In [9]:
df=df[df['FromNodeId']!='#']

In [10]:
df

,FromNodeId,ToNodeId
3,0,11342
4,0,824020
5,0,867923
6,0,891835
7,11342,0
...,...,...
78321,495600,271199
78322,495600,407927
78323,495600,555924
78324,495600,628974


1. Build the page transition matrix A from the data set.

In [11]:
transition_matrix=pd.DataFrame()

In [12]:
transition_matrix = pd.crosstab(df.FromNodeId, df.ToNodeId, dropna=False)

In [13]:
idx = transition_matrix.columns.union(transition_matrix.index)
transition_matrix = transition_matrix.reindex(index = idx, columns=idx, fill_value=0)

In [14]:
transition_matrix

,0,1,10,10009,100150,100238,100352,100459,100467,100553,...,9925,99379,99387,99390,99585,99787,99861,99913,99918,99974
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99787,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
transition_matrix_sum = transition_matrix.sum(axis = 1)
dff=pd.DataFrame(1/10000,index=range(1),columns=transition_matrix.columns)

In [16]:
new_dff=pd.DataFrame(index=range(0),columns=transition_matrix.columns)

2. Identifying dead ends (pages with no out-going links)

In [25]:
# The total number of dead ends in our transistion matrix

(transition_matrix.sum(axis = 1)==0).sum()

1235

3. Make sure that A is a stochastic matrix.

In [26]:
transition_matrix

,0,1,10,10009,100150,100238,100352,100459,100467,100553,...,9925,99379,99387,99390,99585,99787,99861,99913,99918,99974
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99787,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#dead_ends=0
idx_ls=[]
for i in range(0,10000):
    #print(i)
    if(transition_matrix_sum[i]==0):
        #dead_ends=dead_ends+1
        #print(i, cnt)
#       if(transition_matrix_sum[i]==0):
        idx = transition_matrix.index[i]
        idx_ls.append(idx)
        #transition_matrix = transition_matrix.drop(idx)
        new_dff.loc[idx] = dff.loc[0]
        #cnt=cnt+1

In [28]:
new_dff.head()

,0,1,10,10009,100150,100238,100352,100459,100467,100553,...,9925,99379,99387,99390,99585,99787,99861,99913,99918,99974
104074,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
104968,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
105019,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
107010,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
10702,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [30]:
for i in range(0,10000):
    idx = transition_matrix.index[i]
    if(idx not in idx_ls):
        #print(i, cnt)
        transition_matrix.iloc[i]=transition_matrix.iloc[i]/transition_matrix_sum[i]
        #cnt=cnt+1

In [31]:
len(transition_matrix)

10000

In [32]:
transition_matrix=transition_matrix.drop(idx_ls)

In [33]:
new_dff.head()

,0,1,10,10009,100150,100238,100352,100459,100467,100553,...,9925,99379,99387,99390,99585,99787,99861,99913,99918,99974
104074,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
104968,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
105019,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
107010,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
10702,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [34]:
transition_matrix.head()

,0,1,10,10009,100150,100238,100352,100459,100467,100553,...,9925,99379,99387,99390,99585,99787,99861,99913,99918,99974
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
transition_matrix_sum=transition_matrix.sum(axis = 1)

In [36]:
transition_matrix_sum.sum()

8765.0

In [37]:
result=pd.concat([transition_matrix,new_dff])

In [38]:
result=result.sort_index(ascending=True)

In [39]:
#result.to_csv('transition.csv')
result

,0,1,10,10009,100150,100238,100352,100459,100467,100553,...,9925,99379,99387,99390,99585,99787,99861,99913,99918,99974
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# transition_matrixx is confirmed to be a Stochastic matrix
transition_matrixx=result.to_numpy()

4. Implementing the PageRank algorithm with Power Iteration Method, and run it on the transition matrix with a damping factor of d = 0.85 and Eps = 0.001.

In [41]:
def pagerank(A,n=10000,d=0.85,eps=0.001):
    p=pd.DataFrame()
    e=np.ones(n)
    p[0]=np.ones(n)
    p[0]=e/n
    k=1
    while(True):
        #print(k,n,d,eps)
        p[k]=d*np.dot(A.transpose(),p[k-1])+(1-d)*e/n
        if(np.sum(np.abs(p[k]-p[k-1]))<eps):
            break
        k=k+1
    return p[k]

In [42]:
ranks=pagerank(transition_matrixx)

In [43]:
ranks

0       0.000278
1       0.000068
2       0.000213
3       0.000076
4       0.000032
          ...   
9995    0.000034
9996    0.000042
9997    0.000195
9998    0.000041
9999    0.000092
Name: 20, Length: 10000, dtype: float64

In [44]:
output_df=pd.DataFrame(columns=['NodeID','PageRank'])

In [45]:
output_df['NodeID']=result.index
output_df['PageRank']=ranks
output_df

,NodeID,PageRank
0,0,0.000278
1,1,0.000068
2,10,0.000213
3,10009,0.000076
4,100150,0.000032
...,...,...
9995,99787,0.000034
9996,99861,0.000042
9997,99913,0.000195
9998,99918,0.000041


5. Writing the output to a text file. The output format is a two-column: the first column is the web page id and the second column is its PageRank score. Sort the output by descending order of the PageRank scores.

In [46]:
output_df = output_df.sort_values('PageRank',ascending=False)

In [47]:
output_df[:20]

,NodeID,PageRank
4585,486980,0.006931
2278,285814,0.004733
1563,226374,0.003388
767,163075,0.003339
5418,555924,0.002695
2697,32163,0.002385
8679,828963,0.002199
4788,504140,0.002145
3537,396321,0.002118
5924,599130,0.002111


In [54]:
# For verification
#df[df['FromNodeId']=='285814'],len(df[df['FromNodeId']=='285814']), len(df[df['ToNodeId']=='285814'])

In [49]:
output_df.to_csv('output.txt')